In [27]:
import pandas as pd
import numpy as np
import csv
from scipy.signal import find_peaks
from scipy import signal


## Inputs

In [28]:
filepath = 'test_data.csv'
h_cor = 3       # Do not consider first h_vor rows (Header)
Freq = 100      # Sample Frequency [Hz]
freq_r_L = 0.2  # Frequency range Low value [Hz]
freq_r_H = 10   # Frequency range High value [Hz]
dist = 4        # Min distance between peaks - samples between neighbouring peaks (dist * Freq / n = interval_Hz)
limit_frq = 10  # Maximum amount of frequencies to present


## OPEN ACCELEROGRAMS FILE

In [29]:
df0 = []
rcsv0 = []
try:
    ds = open(filepath, 'r', newline='')   
    with ds as file:
        rcsv = csv.reader(file, delimiter = ',')
        for i in range(h_cor):
            next(rcsv)     #ignore header
        for row in rcsv:
            rcsv0.append(pd.to_numeric(row))
        df0 = pd.DataFrame(rcsv0) 
        if len(df0) <= 1:
            raise Exception("File event with issues - Error: it seems empty")
except Exception as e:  
    print("Problems opening event file {} - Error: {}".format(filepath, e))  


## CALCULATE FFT; FIND PEAKS AND CREATE LIST FILE

In [30]:
df0_col_list = []
psd = []
for j in range(len(df0.columns)):                                                                                                                               
    df0_c = df0.iloc[:,j]
    df0_c.reset_index(inplace=True, drop=True)
    n = len(df0_c) 

    # Frequencies
    Ts = 1/Freq  
    frq = np.fft.fftfreq(n=n, d=Ts)     
    frq = frq[range(0,int(n/2))]  
    # Peaks limited between min and max pre-defined range
    for k in range(len(frq)): 
        aux_f = float(frq[k])
        if (aux_f <= freq_r_L): #range LOW
            f_min = k  
        if (aux_f <= freq_r_H): #range HIGH
            f_max = k   
    frq_mm = frq[range(f_min,f_max+1)]                                                  
    # Magnitude
    Y = np.fft.fft(df0_c)/n      
    Y = Y[range(0,int(n/2))]
    Y[0] = 0
    mY = np.abs(Y)                                    
    mY_mm = mY[range(f_min,f_max+1)]  
    # Find max peak
    peakY = np.max(mY_mm)  #max amplitude
    if np.isnan(peakY) == False:
        locY = np.where(mY_mm == peakY)[0][0]  #peak location
    else:
        locY = 0
    frqY = []
    frqY0 = "%.3f" % frq_mm[locY]   #peak frequency value - three decimal places 
    frqY.append(frqY0)                                                                                       
    # Find other peaks
    peaks, _ = find_peaks(mY_mm, distance = dist)
    aux_p = 0
    harm = False
    #List of all peaks - importance descending in steps according to p_peak (percentages from main peak)
    p_peak = [1,0.9,0.8,0.7,0.6,0.5,0.4,0.3]
    for k0 in range(1,len(p_peak)):
        for k in range(0,len(peaks)):           
            frqYz = "%.3f" % frq_mm[peaks[k]]      # Get the actual frequency value  
            #if it larger in the next step of importance, and if it is between range LOW and range HIGH
            if (mY_mm[peaks[k]] >= p_peak[k0]*peakY and mY_mm[peaks[k]] < p_peak[k0-1]*peakY and float(frqYz) >= freq_r_L and float(frqYz) <= freq_r_H):
                #Ignore harmonics and append Frequency value to list
                harm = False
                for f0 in frqY:
                    for hf in range(1,6): #not equal nor one of first 5 harmonics:
                        if (float(frqYz) == float(f0)*hf):
                            harm = True
                if harm == False:        
                    aux_p = aux_p + 1
                    if aux_p <= limit_frq-1:
                        frqY.append(frqYz)                                                
    df0_col_list.append(frqY)            

df1 = pd.DataFrame(df0_col_list).T
print(df1)

try:
    df1.to_csv('result.csv',index=False) 
except Exception as e: 
    print('Problem saving file - Error:', e) 


       0       1       2      3       4       5       6       7       8   \
0  38.950  38.950  14.750  1.600   7.950  32.200  16.800  25.450  12.100   
1  43.000  16.150   2.600  2.400   3.100  22.600  16.300  19.000  38.950   
2   5.200  46.600  13.400   None   3.300  36.400  17.950  19.200  46.600   
3   5.650  24.950   4.900   None   8.450   2.000  16.600  24.600   7.800   
4  16.150    None  11.500   None  13.250  10.400  18.550  31.300   9.850   
5  46.750    None  12.100   None  19.000  14.750  19.100   9.450  10.950   
6   8.500    None  16.100   None  23.300  15.850  15.050  12.300  13.400   
7  11.850    None  20.150   None  42.300  19.000  15.400  18.150  23.300   
8  13.200    None  25.000   None  43.350  23.900  17.000  20.250  43.000   
9  22.000    None  38.950   None  44.000  24.450  18.200  23.300   1.300   

       9   ...      23      24      25      26      27      28      29  \
0  12.100  ...  38.950  16.100  38.950  38.950  38.950  38.950  38.950   
1  38.950  ... 